In [6]:
import os
from covidanalytics import etl as ce
from covidanalytics import analytics as ca
from covidanalytics import plots as cp

In [7]:
reports_path=os.path.join(os.environ['HOME'],'Documents/code/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/')
html_path=os.path.join(os.environ['HOME'],'Documents/code/covid-static/pages/')

In [8]:
param_dict = {'reports_path': reports_path,
          'html': {'dir': html_path + 'usa.html',
                   'img': 'usa.png',
                   'img_shape': 'wide'
                  },
          'country': 'US',
          'start_date': '01/22/2020',
          'end_date': '04/01/2020',
          'title': 'USA - COVID-19'
         }

In [9]:
filenames = ce.dates_to_filenames(param_dict['start_date'], param_dict['end_date'])

In [10]:
recs = []
for f in filenames:
    try:
        f_path = param_dict['reports_path'] + f
        covid = ce.covid_csv_to_df(f_path)
        if 'state' in param_dict.keys():
            daily = ce.covid_province_state(covid, param_dict['state'])
        if 'country' in param_dict.keys():
            daily = ce.covid_country_region(covid, param_dict['country'])
        if 'query' in param_dict.keys():
            daily = ce.covid_query(covid, param_dict['query'])
        recs.append(daily)
    except:
        print(f)


In [11]:
covid_ts = ca.covid_dict_to_timeseries_df(recs, param_dict['start_date'], param_dict['end_date'])


In [12]:
covid_ts

,country_region,confirmed,deaths,active,date,total_change,percent_change
2020-01-22,US,1.0,0.0,1.0,1/22/2020 17:00,0.0,0.000000
2020-01-23,US,1.0,0.0,1.0,1/23/20 17:00,0.0,0.000000
2020-01-24,US,2.0,0.0,2.0,1/24/20 17:00,1.0,100.000000
2020-01-25,US,2.0,0.0,2.0,1/25/20 17:00,0.0,0.000000
2020-01-26,US,5.0,0.0,5.0,1/26/20 16:00,3.0,150.000000
...,...,...,...,...,...,...,...
2020-03-28,US,121478.0,2026.0,118380.0,2020-03-28,19821.0,19.497919
2020-03-29,US,140886.0,2467.0,135754.0,2020-03-29,19408.0,15.976555
2020-03-30,US,161807.0,2978.0,153185.0,2020-03-30,20921.0,14.849595
2020-03-31,US,188172.0,3873.0,177275.0,2020-03-31,26365.0,16.294103


In [2]:
import sklearn
import sklearn.linear_model as lm
import numpy as np

In [24]:
x=np.arange(1,8)
x=x.reshape(-1,1)

In [25]:
y=covid_ts['confirmed'][-8:-1].values
y

array([ 65778.,  83836., 101657., 121478., 140886., 161807., 188172.])

In [36]:
lin_fit=lm.LinearRegression(fit_intercept=True).fit(x, y)

In [32]:
lin_fit.coef_[0]

28691.492857142857

In [39]:
lin_fit.intercept_

43037.28571428574

In [40]:
lin_fit.coef_[0] + lin_fit.intercept_

63121.32142857145

In [41]:
(lin_fit.coef_[0] * 7) + lin_fit.intercept_

183625.53571428568